In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import utils as u
import itertools
import pandas as pd
import cv2

# 1. Read the image, 
###    assign the wanted rotation
###    plot the original and rotated image

In [ ]:
rot_deg=63

root_img_fold = "/home/doga/PycharmProjects/objectLocalization/"
a_image_full_path = os.path.join(root_img_fold, "rarrow.jpg")

img = mpimg.imread(a_image_full_path)
img_rot = u.rot_img_cv(img, rot_deg)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(img)
ax[0].set_title(f"Original image shape {img.shape}", color="white")
ax[1].imshow(img_rot)
ax[1].set_title(f"img_rot rotated by {rot_deg} degrees\nimage shape {img_rot.shape}", color="white")
plt.show()

# 2. Pick a box 
###    now I have img and img_rot
###    define a block to grab and grab it

In [ ]:
#box_corner = {"col_w":230,"row_h":230}; box_wh = {"w":30,"h":40}
#box_corner = {"col_w":230,"row_h":140}; box_wh = {"w":30,"h":40}
box_corner = {"col_w":230,"row_h":95}; box_wh = {"w":30,"h":40}
r = u.get_rectangle((box_corner["col_w"],box_corner["row_h"]), box_wh)
img_box = u.crop_and_show(img, r, figsize=(10, 10), title_color="white")

# 3. Locate the box in original image

In [ ]:
coordinate = u.locate_crop(img, img_box)
print(f"found coordinates = {coordinate}")
print(f"given coordinates = {box_corner}")

# 4. Find center of box in rotated image

In [ ]:
#find the center of the same block if it is rotated
box_def_rot = None #{"box_corner": box_corner, "box_wh":box_wh}
r_rot, bc_rot, bc_wh_rot = u.find_center_of_box_in_rotated_image(rot_deg, img.shape, 
                                                                 rect_def=r, 
                                                                 verbose=1)

# 5. Crop the box from rotated image for search

In [ ]:
#r_rot = u.get_rectangle((bc_rot["col_w"],bc_rot["row_h"]), bc_wh_rot)
img_box_rot= u.crop_and_show(img_rot, r_rot, figsize=(10, 8), title_color="white")

# 6. Locate the cropped image in rotated images

In [ ]:
#%load_ext line_profiler
#from utils import locate_rotated_crop_cosine as lrcc
#%lprun -f  u.locate_rotated_crop_cosine(img, img_box_rot, deg_stride = 5, enforce_cpu=True)
coord_found, codi = u.locate_rotated_crop_cosine(img, img_box_rot, deg_stride = 5, enforce_cpu=False
                                          ,max_w=200, verbose=1)
print(f"found coordinates = {coord_found}")
print(f"given coordinates = {bc_rot}")

# 7. Plot the estimated rotation of image and position of block

In [ ]:
bc_rot_found = {"col_w":coord_found["col_w"],"row_h":coord_found["row_h"]}
r_rot_found = u.get_rectangle((coord_found["col_w"],coord_found["row_h"]), bc_wh_rot)
img_rot_found = u.rot_img_cv(img, coord_found["rot_degree"])
img_box_rot_found = u.crop_and_show(img_rot_found, r_rot_found, figsize=(15, 10), title_color="white")